## Two Datasets are required and both are public named maruti, filled metadata 

In [ ]:
%%capture
!pip install ../input/maruti/facenet_pytorch-2.2.7-py3-none-any.whl
!pip install --no-deps ../input/maruti/maruti-1.3.1-py3-none-any.whl

In [ ]:
from maruti.imports.ml import *

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from facenet_pytorch import MTCNN
from torch.nn.utils.rnn import pack_sequence
test_dir = '../input/deepfake-detection-challenge/test_videos/'
test_videos = sorted(os.listdir(test_dir))
# Any results you write to the current directory are saved as output.

remove this code line it is just so that it won't take much time to commit
START

In [ ]:
test_videos = test_videos[:20]

END
Don't remove anything after this

In [ ]:
## define your version of mtcnn 
mtcnn = MTCNN(select_largest=False,device=device)

In [ ]:
## your model
resnet = torchvision.models.resnet50(False)
resnet.load_state_dict(torch.load('../input/resnet/resnet.pth')) # set path to .pth file
_ = resnet.eval().to(device)

In [ ]:
group_transform = mdata.group_transform['val'] # default transform


#### in case you want your own transform define it here. uncomment following code and define the transform
#### it'll receive a (224,224,3) numpy  image and should perform transforms on that.
# transform = torch_transforms.Compose([
#     torch_transforms.RandomHorizontalFlip()
# ])
## don't change the next line
# group_transform = lambda x: torch.stack(map(transform, x))

In [ ]:
def get_batch(path):
    frame_count, _ = mvis.vid_info(path)
    frame_idx = np.linspace(0, frame_count-1, 8, dtype=int)
    frame_list = list(mvis.get_face_frames(path, frame_idx))
    return frame_list

In [ ]:
def predict(path):
    batch = get_batch(path)
    batch = group_transform(batch).to(device)
    
    with torch.no_grad():
        pred = resnet(batch)
    ## I don't use sigmoid in my model. If you use then remove the next line.
    pred = torch.sigmoid(pred)
    pred = pred.mean().item()
    return pred

In [ ]:
#SPEED TEST
speed_test = True
if speed_test:
    start = time.perf_counter()
    for vid in tqdm(test_videos[:10]):
        print(predict(test_dir+vid))
    print((time.perf_counter()-start)/10)

In [ ]:
start = time.perf_counter()
predictions = []
for i, vid in enumerate(test_videos):
    if i%20==19:
        os.system(f'echo {str(i)} {predictions[-1]:.2f}')
    
    try:
        predictions.append(predict(test_dir+vid))
    except Exception as e:
        print(vid+' error:'+str(e))
        predictions.append(0.5)

print((time.perf_counter()-start)/len(test_videos))

In [ ]:
submission_df = pd.DataFrame({"filename": test_videos, "label": predictions})
submission_df['label'] = submission_df['label'] # use this line for clipping
submission_df.to_csv("submission.csv", index=False)

In [ ]:
## Testing code

import torch.nn.functional as F
from sklearn.metrics import accuracy_score, log_loss
import json


loss_fn = nn.BCELoss()
submission = pd.read_csv('submission.csv')
print(submission.head())

metadata =  maruti.read_json('../input/filled-metadata/filled_metadata.json')

x = submission['filename']
ypred = submission['label'].to_list()
y = []

for i, _ in enumerate(x):
    y.append(float(metadata[x[i]]['label'] == 'FAKE'))
y = torch.tensor(y)
ypred = torch.tensor(ypred)
loss = F.binary_cross_entropy(ypred, y)
assert loss < 0.65, f'Loss Too High: {loss}'
print('loss',F.binary_cross_entropy(ypred, y).item())
if True:
    plt.hist(ypred, bins=200, alpha=0.5)
    plt.hist(y, bins=200, alpha=0.2)
    plt.show()